In [1]:
from ranking import Ranking
from information_retrieval import Embedding_retrieval
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from tqdm.notebook import tqdm

from extract_features_refactored import FeatureExtractor

In [3]:
from get_result import *

In [4]:
df = pd.read_csv('Data/processed_data_v5.csv', sep=';')
df = df[(df.after_dec == True) & (df.tag_disease_covid == True)]

In [5]:
ranking = Ranking('texts')
inforet = Embedding_retrieval()

In [6]:
fe = FeatureExtractor(bert_config_file ="./models/biobert_v1.1_pubmed/bert_config.json",
                      init_checkpoint = "./models/biobert_v1.1_pubmed/model.ckpt-1000000",
                      vocab_file = "./models/biobert_v1.1_pubmed/vocab.txt",
                      batch_size = 32, # Batch size for predictions
                      max_seq_length = 128, # Sequences longer than this will be truncated, and sequences shorter than this will be padded.
                      verbose=0)

In [7]:
query = 'incubation period in days'

In [7]:
path = 'Kaggle COVID-19 medical dictionary.xlsx'
med_dict = pd.read_excel(path, sheet_name=2)

tasks  = med_dict["subtask"].unique()
questions  = med_dict.Question.unique()
subquestions = med_dict.Subquestion.unique()

In [8]:
res_dict = {}
for task in tasks:
    res_dict.update( {task : [{x : [y for y in (med_dict[med_dict.Question==x].Subquestion.unique())]} for x in (med_dict[med_dict["subtask"]==task].Question.unique())]}) 

In [9]:
for i in range(1):
    print('Task {}'.format(i))
    n_task = i
    for n_question, question_dict in enumerate(res_dict[tasks[n_task]]):
        for n_subquestion, subquestion in enumerate(list(question_dict.values())[0]):
            print(subquestion)
            get_answers_files(df, subquestion, ranking, ('BERT', fe), 3, 3, par_k=4, 
                                             inforet_sentence = inforet, only_top_doc = False, 
                                             task = n_task,
                                             question = n_question,
                                             subquestion = n_subquestion)


Task 0
Incubation period
Incubation period by age group


C:\Users\apunt\tfg\clone_test\covid19_challenge\extract_features_refactored.py:444: RuntimeWarning: invalid value encountered in less
  key = lambda x: x[1], reverse=True)[:min(topk, len(score_arr))]


Incubation period by health status
How long are individuals contagious, even after recovery?
Prevalence of asymptomatic shedding and transmission
Prevalence of asymptomatic shedding children
Prevalence of asymptomatic shedding: among hospitalized patients with non-COVID
Geographic variation in transmission by temperature and/or humidity
Particle charge distribution positive negative
Adhesion to hydrophilic phobic surfaces
Susceptibility to environmental cleaning agents
Persistence and stability in nasal discharge
Persistence and stability in sputum
Persistence and stability in urine
Persistence and stability in fecal matter
Persistence and stability in blood
Persistence and stability on aerosols
Persistence and stability on droplets
DOC TOO SHORT
DOC TOO SHORT
DOC TOO SHORT
DOC TOO SHORT
Persistence and stability on copper surfaces
Persistence and stability on stainless steel surfaces
Persistence and stability on plastic
Persistence and stability on reprocessed single-use ventilators
D

In [ ]:
example_dict = get_answers_files(df, query, ranking, ('BERT', fe), 5, 3, par_k=3, inforet_sentence = inforet, only_top_doc = True, name = 'task_0_question_1_subquestion_2')

In [10]:
df.columns

Index(['paper_id', 'title', 'abstract', 'text', 'date', 'authors', 'url',
       'after_dec', 'tag_disease_covid', 'tag_disease_sars',
       'tag_disease_mers', 'tag_disease_corona',
       'tag_design_generic_case_control', 'tag_design_retrospective_cohort',
       'tag_design_cross_sectional_case_control',
       'tag_design_matched_case_control', 'tag_design_prevalence_survey',
       'tag_design_time_series_analysis', 'tag_design_systematic_review',
       'tag_design_randomized_control', 'tag_design_pseudo_randomized_control',
       'tag_design_case_study', 'tag_design_simulation'],
      dtype='object')